# Conjugate Gradient for sparse linear systems

Author: Alexandre Gramfort

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.sparse import linalg as sp_linalg

### Generate simulated data

In [ ]:
np.random.seed(0)
n_samples, n_features = 100, 1
X = np.random.randn(n_samples, n_features)
w = np.random.randn(n_features)
b = 10.
y = np.dot(X, w) + b
y += 0.3 * np.random.randn(n_samples)

In [ ]:
y.shape, X.shape

In [ ]:
X = sparse.csr_matrix(X)  # make X sparse
X

In [ ]:
def sparse_ridge(X, y, alpha=0., x0=None):
    n_features = X.shape[1]
    # matvec = lambda w: X.T.dot(X.dot(w)) + alpha * w
    def matvec(w):
        return X.T.dot(X.dot(w)) + alpha * w
    A = sp_linalg.LinearOperator((n_features, n_features), 
                                 matvec=matvec, dtype=X.dtype)
    w_hat, info = sparse.linalg.cg(A, X.T.dot(y), x0=x0)
    return w_hat

alpha = 0.  # the regularization parameter
w_hat = sparse_ridge(X, y)

In [ ]:
def plot_data(w, b=0.):
    plt.plot(X.toarray()[:, 0], y, 'o', alpha=0.2)

    xx = np.linspace(-2, 2, 100)
    yy = np.dot(xx[:, np.newaxis], w) + b
    plt.plot(xx, yy, 'k')
    plt.grid(True)

plot_data(w_hat)

In [ ]:
Xb = sparse.hstack((X, np.ones((n_samples, 1))))

In [ ]:
Xb.shape

In [ ]:
alpha = 0.  # regularization parameter lambda
w_hat, b_hat = sparse_ridge(Xb, y, alpha=alpha)

In [ ]:
plot_data(w_hat, b_hat)

## Let's do some "Big Data"

In [ ]:
n_samples, n_features = 10000, 1000000
X = sparse.rand(n_samples, n_features, density=0.00001, format='csr')

In [ ]:
X.shape

In [ ]:
X.nnz

In [ ]:
w = np.random.randn(n_features)
y = X.dot(w)

In [ ]:
w_hat = sparse_ridge(X, y, alpha=0.01)

In [ ]:
w_hat.shape

### The benefits of warm start (providing a good init)

In [ ]:
%timeit sparse_ridge(X, y, alpha=0.01, x0=None)

In [ ]:
%timeit sparse_ridge(X, y, alpha=0.02, x0=w_hat)